# Методи оцінки важливості ознак

**Кореляційний аналіз (Pearson / точково-бісеріальна кореляція).** Вимірює лише лінійну залежність між числовою ознакою та бінарною меткою; для бінарного таргету його реалізацією є точково-бісеріальна кореляція. Цей метод не реєструє нелінійні взаємозв’язки і погано працює з категоріальними ознаками (без додаткових перетворень). Він **частково доцільний**: корисний для попередньої фільтрації та перевірки простих лінійних зв’язків, але може пропустити важливі нелінійні ефекти.

**Інформаційна здобутість / взаємна інформація (Information Gain / Mutual Information).** Оцінює скільки інформації дає ознака про ціль, використовуючи ентропію чи взаємну інформацію. Підходить для числових і дискретних ознак (для числових потрібне дискретизування або ядрове оцінювання) і здатен фіксувати як лінійні, так і нелінійні залежності з ціллю. Зазвичай використовується як фільтровий метод. Загалом **доцільний**: неупереджений до типу даних, але результат може бути нестабільним при складних перетинах ознак (за відсутності урахування взаємодій між ознаками).

**ANOVA (F-тест) для бінарної цілі.** Перевіряє, чи різняться середні значення числової ознаки між класами. Фактично це статистичний аналог точково-бісеріальної кореляції. Метод **частково доцільний**: він швидкий і зрозумілий для числових ознак із приблизно нормальним розподілом, але оцінює лише лінійну (середню) різницю і не спіймає нестандартних чи нелінійних залежностей. Також не застосовується до нечислових ознак без перекодування.

**χ²-тест для незалежності.** Розраховується для категоріальних ознак (побудова таблиці спряженості з бінарною ціллю) – виявляє статистичну асоціацію. Цей метод **частково доцільний**: добре підходить для явно категоріальних полів і фіксує тільки незалежність/залежність, але не вказує на форму впливу і не застосовується до числових без дискретизації. Як і інші фільтри, χ²-тест ігнорує складні взаємодії ознак.

**Лінійна логістична регресія (коефіцієнти).** Лінійна модель із бінарною ціллю, де величина коефіцієнта (за умови стандартизації ознак) може слугувати мірою важливості. Цей **вбудований** метод (embedded) дозволяє оцінити внесок кожної ознаки одночасно з врахуванням інших. Логістична регресія **частково доцільна** для нашої задачі: вона враховує взаємодію всіх ознак через модель, але фіксує тільки лінійний зв’язок (пряма гіпотеза в логіті). Якщо зв’язок між ознакою та ціллю складний (мінливий/нелінійний), логістична регресія може недооцінити її важливість. До того ж вибір скейлінгу впливає на величину коефіцієнтів. У випадку корельованих ознак логістична регресія часто довільно розподіляє вагу між ними (утримуючи одну ігнорує іншу) ([Feature importance — Scikit-learn course](https://inria.github.io/scikit-learn-mooc/python_scripts/dev_features_importance.html#:~:text=We%20can%20see%20that%20out,models%2C%20as%20they%20are%20imperfect)), тому її інтерпретація в таких випадках неточна.

**Дерева рішень (Random Forest, Gradient Boosting) – імпуріті-важливість.** Ансамблеві методи на основі дерев (Bagging чи Boosting) можуть оцінити важливість ознак через зменшення метрики нечистоти (наприклад, Gini чи ентропії) при розбитті. Random Forest використовує **середній коефіцієнт зменшення нечистоти** по всіх деревах як імпуті-фактор важливості. Такий підхід **доцільний**: він природно обробляє числові й категоріальні ознаки та захоплює складні і нелінійні залежності (через структуру дерева). Проте він може бути упередженим. Зокрема, Breiman у Random Forest згадує використання внутрішніх оцінок моделі для вимірювання важливості ознак ([Random Forests | Machine Learning
        ](https://link.springer.com/article/10.1023/A:1010933404324#:~:text=of%20the%20Thirteenth%20International%20conference%2C,are%20also%20used%20to%20measure)). Водночас Штрёбл та ін. (2007) зауважують, що імпуріті-важливість (Mean Decrease Impurity) може віддавати перевагу ознакам із більшим числом рівнів чи діапазоном ([Bias in random forest variable importance measures: illustrations, sources and a solution - PubMed](https://pubmed.ncbi.nlm.nih.gov/17254353/#:~:text=variable%20importance%20measures%20are%20used,individual%20classification%20trees%20used%20to)). Тобто категоріальні ознаки з багатьма категоріями та ознаки з широким шкалюванням штучно виглядатимуть важливішими. У підсумку ансамблеві деревні методи варто використовувати, але з обережністю: наприклад, додатково оцінювати значимість через зовнішні процедури.

**Пермутаційна важливість (Permutation Feature Importance).** Модел-незалежний (model-agnostic) метод: по черзі перемішуємо значення кожної ознаки в тестовому наборі і дивимося, наскільки погіршилась робоча точність моделі. Якщо перестановка конкретної ознаки суттєво знижує якість, вона вважається важливою. Цей метод **доцільний** для нашої задачі: він враховує будь-які нелінійні ефекти в межах обраного алгоритму і працює з усіма типами даних без зміни моделі. Пермутація не має вад, притаманних MDI; як зазначено в документації sklearn, вона не упереджена щодо кількості категорій і охоплює корельовані ознаки коректніше ([Feature importances with a forest of trees — scikit-learn 1.6.1 documentation](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#:~:text=Permutation%20feature%20importance%20overcomes%20limitations,out%20test%20set)). Проте її недолік – висока обчислювальна вартість (потрібно повторно оцінювати модель) і залежність від обраної моделі. Якщо модель погано відображає залежності, пермутаційна важливість може бути неправильною.

**Shapley-значення (SHAP).** Сучасний метод, що базується на ігрово-теоретичних SHAP-відіграшах: призначає кожній ознаці індивідуальне значення внеску у прогноз (ціль може братись як зі всього датасету, так і для кожного прикладу окремо). SHAP **доцільний** та популярний: це модель-незалежний метод з теоретично обґрунтованою унікальною мірою важливості ([[1705.07874] A Unified Approach to Interpreting Model Predictions](https://arxiv.org/abs/1705.07874#:~:text=unified%20framework%20for%20interpreting%20predictions%2C,with%20human%20intuition%20than%20previous)). Він виявляє нелінійні взаємодії й коректно реагує на корельовані ознаки. Як вказує огляд, SHAP є «візуально інформативним та застосовним до будь-якого типу даних» ([Explainable AI: A Review of Machine Learning Interpretability Methods](https://www.mdpi.com/1099-4300/23/1/18#:~:text=In%20terms%20of%20explaining%20any,to%20any%20type%20of%20data)). Недоліком є велика витрата обчислювальних ресурсів (особливо KernelSHAP) та складність інтерпретації для багатьох ознак. У цілому SHAP є дуже потужним засобом для глобального ранжування ознак (високі модульовані значення Shapley) і рекомендований для використання.

**LIME.** Метод, що навчає локальну лінійну модель (сурогат) у околі конкретного прогнозу, і через це показує, які ознаки зірвно вплинули на це рішення. **Частково доцільний** для аналізу: LIME є модель-незалежним і гнучким ([Explainable AI: A Review of Machine Learning Interpretability Methods](https://www.mdpi.com/1099-4300/23/1/18#:~:text=In%20terms%20of%20explaining%20any,to%20any%20type%20of%20data)), але він дає здебільшого **локальне** пояснення (індивідуально для кожного замовлення) і не формує глобального рейтингу ознак. Наявність рандомізованих кроків може зменшувати відтворюваність, а серія локальних моделей потребує додаткових спроб, щоб узагальнити на всю вибірку. Тому LIME більше підходить для розуміння окремих прогнозів, але **не є найкращим методом** прямого ранжування ознак у класифікації.

**Графік часткової залежності (PDP).** Візуальний метод: показує, як зміна однієї ознаки впливає на прогноз моделі при усередненні за іншими. PDP сам по собі не видає числової оцінки важливості, але дає уявлення про форму впливу (лінійна чи складна). Він **частково доцільний** для нашої задачі: добре відображає нелінійні ефекти і працює з будь-якими моделями, проте є важким у розрахунку та інтерпретації при великій кількості ознак і не дає однозначного ранжування. Як зауважено в огляді методів інтерпретації, PDP лишається популярним засобом візуалізації відносин у даних ([Explainable AI: A Review of Machine Learning Interpretability Methods](https://www.mdpi.com/1099-4300/23/1/18#:~:text=In%20terms%20of%20explaining%20any,to%20any%20type%20of%20data)), але заради чисто числової оцінки важливості він обмежений.

**Wrapper-методи (RFE, Boruta тощо).** Наприклад, Recursive Feature Elimination (RFE) послідовно видаляє найменш важливі ознаки, вимірюючи при цьому зміну якості моделі. Такі методи використовують модель як «чорний ящик» і оцінюють сукупний вплив наборів ознак (як описано у Guyon та Elisseeff 2003) ([guyon03a.dvi](https://www.jmlr.org/papers/volume3/guyon03a/guyon03a.pdf#:~:text=amounts%20of%20computation%2C%20but%20it,strategies%20does%20not%20necessarily%20mean)). Wrapper-методи **частково доцільні**: вони здатні враховувати взаємодії ознак (в залежності від обраного базового алгоритму) і, зазвичай, дають добре підібраний підмножину ознак. Але вони дуже затратні обчислювально та можуть давати нестійкі результати (залежно від початкової оцінки та стабільності моделі). У нашій задачі такі методи можуть допомогти при остаточній перевірці відбору ознак, але не підходять як швидкий спосіб отримати індивідуальні оцінки.

**Вбудовані методи (L1-регуляризація, Lasso).** Методи на кшталт L1-логістичної регресії одночасно навчають модель і роблять селекцію ознак (обнуляючи неважливі коефіцієнти). Це **частково доцільний** підхід: він автоматично оцінює важливість в контексті логістичної моделі і враховує корельовані ознаки. Проте, як і звичайна логістика, захоплює тільки лінійні зв’язки, а відбір ознак при наявності сильно корельованих полів може бути довільним і приховувати справжню важливість. Такий метод радше підходить для зменшення розмірності і швидкого прототипування, ніж для детального ранжування ознак.



# Дублікати і подібні методи

- **Лінійні міри взаємозв’язку (кореляція, ANOVA, логістичні коефіцієнти).** Всі вони базуються на припущенні лінійності: точково-бісеріальна кореляція і F-тест ANOVA по суті оцінюють різницю середніх між двома класами (див. ([guyon03a.dvi](https://www.jmlr.org/papers/volume3/guyon03a/guyon03a.pdf#:~:text=the%20variability%20of%20the%20variable,Hur%20and%20Guyon))), а коефіцієнти лінійної чи логістичної регресії теж відображають залежність першого порядку. Усі ці методи можуть *взаємозамінно* фільтрувати ознаки, але вони різняться деталями (кореляція – чиста числова зв’язність, ANOVA – порівняння дисперсії, регресія – узгодження в моделі). Вони подібні за математикою (по суті аналіз дисперсії чи коваріації), але по-різному рахують рівняння.

- **Методи на основі дерев (Random Forest vs Gradient Boosting).** Обидва підходи використовують деревні спліт-функції та критерії нечистоти (Gini, ентропія) для оцінки важливості. Різниця в математиці: Random Forest – це ансамбль незалежних випадкових дерев (bagging), тоді як Gradient Boosting – послідовна жадібна побудова дерев. Обидва видають сумарні показники важливості (середнє зменшення нечистоти, сумарний приріст). Таким чином за змістом методи дуже схожі, але реалізують різні стратегії навчання (що впливає на розподіл важливостей і стабільність).

- **Пермутаційна важливість vs Mean Decrease Accuracy (OOB-оцінка в Random Forest).** Це фактично дві сторони однієї концепції: оцінювати важливість через зміну метрики моделі при «вимиканні» ознаки. У Per­mutation importance ми вручну перемішуємо стовпець, а в класичній Random Forest є внутрішня оцінка «mean decrease accuracy» на OOB-вибірці. Технологічно це дуже схожий підхід: у підсумку обидва оцінюють, наскільки упавлисть точність без даних цієї ознаки.

- **LIME vs KernelSHAP (адитивні локальні сурогати).** Обидва методи навчають просту локальну модель (лінійну функцію) навколо конкретного прикладу, щоб пояснити внесок ознак. LIME випадково вибирає набір сусідніх точок, а KernelSHAP використовує спеціальні ваги, що відповідають задачі Шеплі. Обидва прагнуть локального “розкладу” прогнозу на компоненти. Подібність у тому, що і там, і там ми отримуємо локальні коефіцієнти, але різниця – у формуванні вибірки і гарантіях: SHAP гарантує адитивність і консистентність пояснень, тоді як LIME – більш евристичний і залежить від параметрів вибору сусідів.

- **Інформаційно-теоретичні методи (Information Gain vs Mutual Information vs χ²).** Всі троє – це міри залежності між ознакою та меткою. Information Gain (часто використовується при побудові дерев) – це по суті взаємна інформація між ознакою та ціллю. Mutual Information (загальний) вимірює відсутність незалежності (ненульові значення означають залежність). χ²-тест також перевіряє залежність через таблицю спряженості. Тобто всі вони подібні за ідеєю – знаходять статистичну зв’язність, – але відрізняються обчисленням: χ² дає тестову статистику, а інформація дає кількість «ентропії», що пояснюється ознакою.

- **Wrapper-методи (RFE, Boruta тощо) та вбудовані (L1-регуляризація).** RFE – це по суті один із способів реалізувати wrapper-підхід: він використовує модель для поетапної відбраковки ознак. Boruta теж за концепцією покладається на випадковий ліс із додатковими «тіньовими» ознаками. За своєю математикою це різні процедури селекції, але їх усіх можна об’єднати в одну категорію «використовують модель як чорний ящик для селекції». L1-регуляризація є embedded-методом, де відбіраються ознаки за нульовими коефіцієнтами. Усі ці підходи відрізняються тільки тим, як саме відбувається ітерація відбору – але в основі лежить одна і та ж ідея: повторно тренувати модель і дивитися, що з ознаками відбувається.

# Висновки

Найбільш **доцільні** методи для нашої задачі класифікації – це ті, що враховують складні нелінійні зв’язки та працюють з різними типами даних. До них відносимо ансамблеві методи на основі дерев (Random Forest, Gradient Boosting), пермутаційну важливість та Shapley-аналіз (SHAP). Ці підходи забезпечують коректне ранжування ознак без обмеження на лінійність чи тип даних і здатні виявити взаємодії між ознаками ([Random Forests | Machine Learning
        ](https://link.springer.com/article/10.1023/A:1010933404324#:~:text=of%20the%20Thirteenth%20International%20conference%2C,are%20also%20used%20to%20measure)) ([Explainable AI: A Review of Machine Learning Interpretability Methods](https://www.mdpi.com/1099-4300/23/1/18#:~:text=In%20terms%20of%20explaining%20any,to%20any%20type%20of%20data)). Для побудови інтерпретованих моделей можна використовувати логістичну регресію (адже вона дає зрозумілі коефіцієнти), але треба пам’ятати, що вона зловить тільки лінійні ефекти. Інформаційно-теоретичні фільтри (mutual information, інформаційний приріст, χ²) дають швидкий огляд важливості, але вимагають обережності: вони дають лише узагальнені оцінки асоціативності без урахування спільного впливу ознак.

Методи типу LIME і PDP варто застосовувати вже для пояснення конкретних моделей чи прогнозів: вони добре ілюструють вплив ознак, але самі по собі не видають простого числового ранжиру. F-тест і кореляція корисні для попередньої фільтрації, але, зважаючи на потенційні нелінійні залежності в даних, на них спиратися як на єдину міру важливості не слід. Таким чином, **комплексний підхід** – наприклад, поєднання деревних методів та пермутаційних оцінок (або SHAP) – дасть найбільш повну картину індивідуального внеску ознак у класифікацію заказів у B2B.

**Джерела:** результати досліджень і синтез методів інтерпретації моделей та важливості ознак ([Random Forests | Machine Learning
        ](https://link.springer.com/article/10.1023/A:1010933404324#:~:text=of%20the%20Thirteenth%20International%20conference%2C,are%20also%20used%20to%20measure)) ([Bias in random forest variable importance measures: illustrations, sources and a solution - PubMed](https://pubmed.ncbi.nlm.nih.gov/17254353/#:~:text=variable%20importance%20measures%20are%20used,individual%20classification%20trees%20used%20to)) ([Explainable AI: A Review of Machine Learning Interpretability Methods](https://www.mdpi.com/1099-4300/23/1/18#:~:text=In%20terms%20of%20explaining%20any,to%20any%20type%20of%20data)) ([[1705.07874] A Unified Approach to Interpreting Model Predictions](https://arxiv.org/abs/1705.07874#:~:text=unified%20framework%20for%20interpreting%20predictions%2C,with%20human%20intuition%20than%20previous)) ([“Why Should I Trust You?”: Explaining the Predictions of Any Classifier - ACL Anthology](https://aclanthology.org/N16-3020/#:~:text=10.18653%2Fv1%2FN16,pages%2097%E2%80%93101%2C%20San%20Diego%2C%20California)).